<a href="https://colab.research.google.com/github/JunwenFang/3001ML_in_Finance/blob/main/3001_default_prediction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import pandas as pd
import numpy as np
import os
import sys
import matplotlib.pyplot as plt
import random
from datetime import datetime
from collections import Counter
import itertools
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.utils.data import TensorDataset, DataLoader
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from sklearn.model_selection import train_test_split
# import gluonts
# from gluonts.dataset.common import ListDataset
# import optuna
# from gluonts.evaluation import Evaluator
# from gluonts.evaluation.backtest import make_evaluation_predictions
import yfinance as yf
import warnings
warnings.filterwarnings('ignore')
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"  # 每个代码块都能输出结果
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [4]:
data = pd.read_csv("train_demo_subset.csv")
data.columns
data

Index(['Unnamed: 0', 'id', 'stmt_date', 'HQ_city', 'legal_struct',
       'ateco_sector', 'def_date', 'fs_year', 'asst_intang_fixed',
       'asst_tang_fixed', 'asst_fixed_fin', 'asst_current', 'AR',
       'cash_and_equiv', 'asst_tot', 'eqty_tot', 'eqty_corp_family_tot',
       'liab_lt', 'liab_lt_emp', 'debt_bank_st', 'debt_bank_lt', 'debt_fin_st',
       'debt_fin_lt', 'AP_st', 'AP_lt', 'debt_st', 'debt_lt', 'rev_operating',
       'COGS', 'prof_operations', 'goodwill', 'inc_financing', 'exp_financing',
       'prof_financing', 'inc_extraord', 'taxes', 'profit', 'days_rec',
       'ebitda', 'roa', 'roe', 'wc_net', 'margin_fin', 'cf_operations'],
      dtype='object')

,Unnamed: 0,id,stmt_date,HQ_city,legal_struct,ateco_sector,def_date,fs_year,asst_intang_fixed,asst_tang_fixed,...,inc_extraord,taxes,profit,days_rec,ebitda,roa,roe,wc_net,margin_fin,cf_operations
0,1630,36220135,12/31/07,108,SAU,28,12/31/99,2007,16159,2643651,...,-1024,2623921,2172169,NaN,1371337,17.31,36.57,2014680,2369981.0,2419059
1,2178,40700775,12/31/07,77,SRL,23,12/31/99,2007,30150,6087428,...,-407687,285969,86481,-46.69,2083697,3.87,0.85,4537122,3995928.0,1424877
2,430,11540689,12/31/07,68,SPA,10,12/31/99,2007,257440,1581628,...,-56413,-54252,-116042,242.38,404853,3.69,-12.95,463308,-1088326.0,15854
3,2156,40600579,12/31/07,57,SPA,10,12/31/99,2007,36711,1199476,...,-3995,9653,-24856,196.66,178873,2.95,-3.11,171866,-435927.0,67851
4,843,21000161,12/31/07,97,SPA,25,12/31/99,2007,32557,936821,...,6564,270833,83815,50.41,452830,3.14,2.95,2761349,1874079.0,313063
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
995,4144739,9639771006,12/31/12,58,SRL,62,12/31/99,2012,72118,14625,...,20290,292001,188479,NaN,398043,9.14,51.41,-428110,-734020.0,251418
996,1492115,2119590418,12/31/12,41,SRL,68,12/31/99,2012,0,3297295,...,1,-1821261,-1823560,NaN,64986,0.12,-64.87,-360259,-487159.0,-1762717
997,4144079,9634430012,12/31/12,1,SRL,41,12/31/99,2012,4302,452,...,83126,18497,11376,NaN,-30050,-1.24,1.63,1670171,693166.0,13533
998,4142262,9613150011,12/31/12,1,SRL,68,12/31/99,2012,381136,195649,...,-26231,-25556,73,NaN,98836,1.02,0.00,971233,1139363.0,34784


# EDA

In [6]:
data.describe(include=['object'])
data.dtypes
data.isnull().sum()

,stmt_date,legal_struct,def_date
count,1000,1000,1000
unique,6,5,13
top,12/31/07,SRL,12/31/99
freq,241,695,907


,0
Unnamed: 0,int64
id,int64
stmt_date,object
HQ_city,int64
legal_struct,object
ateco_sector,int64
def_date,object
fs_year,int64
asst_intang_fixed,int64
asst_tang_fixed,int64


,0
Unnamed: 0,0
id,0
stmt_date,0
HQ_city,0
legal_struct,0
ateco_sector,0
def_date,0
fs_year,0
asst_intang_fixed,0
asst_tang_fixed,0


# Model